# Stablecoin Depeg Prediction - Descriptive Analytics

Comprehensive exploratory data analysis of USDT and USDC stablecoin data from multiple sources:
- **Binance**: Trading data (OHLCV, spread, buy pressure)
- **DefiLlama**: Supply metrics
- **CoinGecko**: Direct USD prices
- **Fear & Greed Index**: Market sentiment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
colors = {'usdt': '#26A17B', 'usdc': '#2775CA'}  # Official brand colors

# Paths
PROJECT_ROOT = Path('.').resolve().parent
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
RAW_DIR = PROJECT_ROOT / 'data' / 'raw'

---
## 1. Data Overview

In [ ]:
# Load data
df = pd.read_csv(PROCESSED_DIR / 'combined_stablecoins_daily.csv')
df['date'] = pd.to_datetime(df['date'])

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Duration: {(df['date'].max() - df['date'].min()).days:,} days")
print(f"\nCoins: {df['coin'].unique().tolist()}")
print(f"USDT records: {len(df[df['coin']=='usdt']):,}")
print(f"USDC records: {len(df[df['coin']=='usdc']):,}")

In [ ]:
# Column information
print("\nCOLUMN DETAILS:")
print("-"*60)
col_info = pd.DataFrame({
    'dtype': df.dtypes,
    'non_null': df.notna().sum(),
    'null_pct': (df.isna().sum() / len(df) * 100).round(1),
    'unique': df.nunique()
})
print(col_info.to_string())

In [ ]:
# Sample data
print("\nSAMPLE DATA (last 5 rows):")
df.tail()

---
## 2. Summary Statistics

In [ ]:
# Key numeric columns
key_cols = ['implied_price', 'close', 'quote_volume', 'spread_proxy', 
            'buy_ratio', 'total_circulating', 'fear_greed_value']

# Stats by coin
for coin in ['usdt', 'usdc']:
    print(f"\n{'='*60}")
    print(f"{coin.upper()} SUMMARY STATISTICS")
    print("="*60)
    coin_df = df[df['coin'] == coin][key_cols]
    display(coin_df.describe().T.round(4))

In [ ]:
# Price stability metrics
print("\nPRICE STABILITY METRICS")
print("-"*60)

stability_stats = []
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin]
    deviation = coin_df['implied_price'] - 1.0
    
    stats_dict = {
        'Coin': coin.upper(),
        'Mean Deviation': f"{deviation.mean()*100:.4f}%",
        'Std Deviation': f"{deviation.std()*100:.4f}%",
        'Min (worst negative)': f"{deviation.min()*100:.4f}%",
        'Max (worst positive)': f"{deviation.max()*100:.4f}%",
        'Median': f"{deviation.median()*100:.4f}%",
        'Skewness': f"{deviation.skew():.4f}",
        'Kurtosis': f"{deviation.kurtosis():.4f}",
        'Days > 0.5%': (deviation.abs() > 0.005).sum(),
        'Days > 1.0%': (deviation.abs() > 0.01).sum(),
    }
    stability_stats.append(stats_dict)

pd.DataFrame(stability_stats).set_index('Coin').T

---
## 3. Missing Data Analysis

In [ ]:
# Missing data heatmap
fig, ax = plt.subplots(figsize=(14, 6))

missing = df.isna()
sns.heatmap(missing.T, cbar=True, yticklabels=True, cmap='YlOrRd', ax=ax)
ax.set_title('Missing Data Pattern (Yellow = Missing)', fontsize=14)
ax.set_xlabel('Row Index')
plt.tight_layout()
plt.show()

# Missing data summary
missing_summary = df.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    print("\nColumns with missing values:")
    for col, count in missing_summary.items():
        print(f"  {col}: {count:,} ({count/len(df)*100:.1f}%)")
else:
    print("\nNo missing values in core columns!")

In [ ]:
# CoinGecko data availability (only last 365 days)
if 'cg_price' in df.columns:
    cg_available = df[df['cg_price'].notna()]
    print(f"\nCoinGecko Data Availability:")
    print(f"  Records with CG data: {len(cg_available):,} / {len(df):,}")
    print(f"  Date range: {cg_available['date'].min().date()} to {cg_available['date'].max().date()}")

---
## 4. Price Analysis

In [ ]:
# Price deviation time series
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

for ax, coin in zip(axes, ['usdt', 'usdc']):
    coin_df = df[df['coin'] == coin].set_index('date')
    deviation = (coin_df['implied_price'] - 1) * 100
    
    ax.fill_between(deviation.index, deviation, 0, 
                    where=deviation >= 0, color=colors[coin], alpha=0.3, label='Above peg')
    ax.fill_between(deviation.index, deviation, 0, 
                    where=deviation < 0, color='red', alpha=0.3, label='Below peg')
    ax.plot(deviation.index, deviation, color=colors[coin], linewidth=0.5)
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax.axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='±0.5% threshold')
    ax.axhline(y=-0.5, color='orange', linestyle='--', alpha=0.7)
    
    ax.set_ylabel('Deviation (%)')
    ax.set_title(f'{coin.upper()} Price Deviation from $1.00', fontsize=12)
    ax.legend(loc='upper right')
    ax.set_ylim(-2, 2)

axes[1].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of deviations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Histogram
ax1 = axes[0]
for coin in ['usdt', 'usdc']:
    deviation = (df[df['coin'] == coin]['implied_price'] - 1) * 100
    ax1.hist(deviation, bins=100, alpha=0.6, label=coin.upper(), color=colors[coin])
ax1.axvline(x=0, color='red', linestyle='--')
ax1.set_xlabel('Price Deviation (%)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Price Deviations')
ax1.legend()
ax1.set_xlim(-1, 1)

# Box plot
ax2 = axes[1]
data_to_plot = [((df[df['coin'] == 'usdt']['implied_price'] - 1) * 100).values,
                ((df[df['coin'] == 'usdc']['implied_price'] - 1) * 100).values]
bp = ax2.boxplot(data_to_plot, labels=['USDT', 'USDC'], patch_artist=True)
bp['boxes'][0].set_facecolor(colors['usdt'])
bp['boxes'][1].set_facecolor(colors['usdc'])
ax2.axhline(y=0, color='red', linestyle='--')
ax2.set_ylabel('Price Deviation (%)')
ax2.set_title('Deviation Box Plot')

# QQ plot for normality
ax3 = axes[2]
for coin in ['usdt', 'usdc']:
    deviation = (df[df['coin'] == coin]['implied_price'] - 1)
    stats.probplot(deviation, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normality Check)')
ax3.get_lines()[0].set_color(colors['usdt'])
ax3.get_lines()[1].set_color(colors['usdt'])

plt.tight_layout()
plt.show()

In [ ]:
# Compare implied price vs CoinGecko direct price
if 'cg_price' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for ax, coin in zip(axes, ['usdt', 'usdc']):
        coin_df = df[(df['coin'] == coin) & (df['cg_price'].notna())].copy()
        
        ax.scatter(coin_df['implied_price'], coin_df['cg_price'], 
                   alpha=0.5, s=10, color=colors[coin])
        ax.plot([0.99, 1.01], [0.99, 1.01], 'r--', label='Perfect agreement')
        
        # Calculate correlation
        corr = coin_df['implied_price'].corr(coin_df['cg_price'])
        ax.text(0.05, 0.95, f'Correlation: {corr:.4f}', transform=ax.transAxes, 
                fontsize=10, verticalalignment='top')
        
        ax.set_xlabel('Implied Price (from BTC pair)')
        ax.set_ylabel('CoinGecko Direct Price')
        ax.set_title(f'{coin.upper()}: Implied vs Direct Price')
        ax.legend()
        ax.set_xlim(0.99, 1.01)
        ax.set_ylim(0.99, 1.01)
    
    plt.tight_layout()
    plt.show()

---
## 5. Volume & Trading Activity

In [ ]:
# Volume over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Daily volume
ax1 = axes[0]
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].set_index('date')
    ax1.fill_between(coin_df.index, coin_df['quote_volume']/1e9, 
                     alpha=0.5, label=coin.upper(), color=colors[coin])
ax1.set_ylabel('Daily Volume ($ Billions)')
ax1.set_title('Trading Volume Over Time')
ax1.legend()

# 30-day rolling average
ax2 = axes[1]
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].set_index('date')
    rolling_vol = coin_df['quote_volume'].rolling(30).mean() / 1e9
    ax2.plot(rolling_vol.index, rolling_vol, label=f'{coin.upper()} (30d MA)', 
             color=colors[coin], linewidth=2)
ax2.set_ylabel('Volume ($ Billions, 30d MA)')
ax2.set_xlabel('Date')
ax2.set_title('Smoothed Trading Volume')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Volume statistics
print("VOLUME STATISTICS ($ Millions)")
print("-"*60)
for coin in ['usdt', 'usdc']:
    vol = df[df['coin'] == coin]['quote_volume'] / 1e6
    print(f"\n{coin.upper()}:")
    print(f"  Mean daily volume:   ${vol.mean():,.0f}M")
    print(f"  Median daily volume: ${vol.median():,.0f}M")
    print(f"  Max daily volume:    ${vol.max():,.0f}M")
    print(f"  Total volume:        ${vol.sum()/1000:,.0f}B")

---
## 6. Supply Analysis

In [ ]:
# Supply over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Total supply
ax1 = axes[0]
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].set_index('date')
    ax1.plot(coin_df.index, coin_df['total_circulating']/1e9, 
             label=coin.upper(), color=colors[coin], linewidth=2)
ax1.set_ylabel('Circulating Supply ($ Billions)')
ax1.set_title('Stablecoin Supply Growth')
ax1.legend()

# Daily supply change
ax2 = axes[1]
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].set_index('date')
    ax2.plot(coin_df.index, coin_df['circulating_change_pct']*100, 
             label=coin.upper(), color=colors[coin], alpha=0.7, linewidth=0.8)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('Daily Supply Change (%)')
ax2.set_xlabel('Date')
ax2.set_title('Daily Supply Changes')
ax2.set_ylim(-5, 5)
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Supply growth statistics
print("SUPPLY GROWTH ANALYSIS")
print("-"*60)
for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].sort_values('date')
    start_supply = coin_df['total_circulating'].iloc[0] / 1e9
    end_supply = coin_df['total_circulating'].iloc[-1] / 1e9
    growth = (end_supply / start_supply - 1) * 100
    
    print(f"\n{coin.upper()}:")
    print(f"  Starting supply: ${start_supply:.2f}B")
    print(f"  Ending supply:   ${end_supply:.2f}B")
    print(f"  Total growth:    {growth:.1f}%")
    print(f"  CAGR:            {((end_supply/start_supply)**(365/len(coin_df))-1)*100:.1f}%")

---
## 7. Fear & Greed Index Analysis

In [ ]:
if 'fear_greed_value' in df.columns:
    # Get unique fear/greed values (one per day, not per coin)
    fg_df = df[['date', 'fear_greed_value', 'fear_greed_class']].drop_duplicates().dropna()
    fg_df = fg_df.set_index('date').sort_index()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Time series
    ax1 = axes[0, 0]
    ax1.fill_between(fg_df.index, fg_df['fear_greed_value'], 50,
                     where=fg_df['fear_greed_value'] >= 50, color='green', alpha=0.3)
    ax1.fill_between(fg_df.index, fg_df['fear_greed_value'], 50,
                     where=fg_df['fear_greed_value'] < 50, color='red', alpha=0.3)
    ax1.plot(fg_df.index, fg_df['fear_greed_value'], color='black', linewidth=0.5)
    ax1.axhline(y=50, color='gray', linestyle='--')
    ax1.axhline(y=25, color='red', linestyle=':', alpha=0.5, label='Extreme Fear')
    ax1.axhline(y=75, color='green', linestyle=':', alpha=0.5, label='Extreme Greed')
    ax1.set_ylabel('Fear & Greed Index')
    ax1.set_title('Crypto Fear & Greed Index Over Time')
    ax1.set_ylim(0, 100)
    ax1.legend()
    
    # Distribution
    ax2 = axes[0, 1]
    ax2.hist(fg_df['fear_greed_value'], bins=20, color='purple', alpha=0.7, edgecolor='black')
    ax2.axvline(x=fg_df['fear_greed_value'].mean(), color='red', linestyle='--', 
                label=f'Mean: {fg_df["fear_greed_value"].mean():.1f}')
    ax2.axvline(x=fg_df['fear_greed_value'].median(), color='blue', linestyle='--',
                label=f'Median: {fg_df["fear_greed_value"].median():.1f}')
    ax2.set_xlabel('Fear & Greed Value')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Sentiment')
    ax2.legend()
    
    # Class breakdown
    ax3 = axes[1, 0]
    class_counts = fg_df['fear_greed_class'].value_counts()
    class_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
    class_counts = class_counts.reindex([c for c in class_order if c in class_counts.index])
    colors_bar = ['darkred', 'red', 'gray', 'lightgreen', 'green']
    ax3.bar(class_counts.index, class_counts.values, color=colors_bar[:len(class_counts)])
    ax3.set_ylabel('Number of Days')
    ax3.set_title('Sentiment Classification Distribution')
    plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Fear/Greed vs Price Deviation
    ax4 = axes[1, 1]
    merged = df[df['fear_greed_value'].notna()].copy()
    merged['price_dev'] = (merged['implied_price'] - 1).abs() * 100
    ax4.scatter(merged['fear_greed_value'], merged['price_dev'], 
                alpha=0.3, s=5, c=merged['coin'].map(colors))
    ax4.set_xlabel('Fear & Greed Index')
    ax4.set_ylabel('Absolute Price Deviation (%)')
    ax4.set_title('Sentiment vs Price Deviation')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("\nFEAR & GREED STATISTICS")
    print("-"*40)
    print(f"Mean:   {fg_df['fear_greed_value'].mean():.1f}")
    print(f"Median: {fg_df['fear_greed_value'].median():.1f}")
    print(f"Std:    {fg_df['fear_greed_value'].std():.1f}")
    print(f"Min:    {fg_df['fear_greed_value'].min():.0f}")
    print(f"Max:    {fg_df['fear_greed_value'].max():.0f}")

---
## 8. Correlation Analysis

In [ ]:
# Select features for correlation
corr_cols = [
    'implied_price', 'close', 'quote_volume', 'spread_proxy', 'buy_ratio',
    'total_circulating', 'circulating_change_pct', 'fear_greed_value'
]
corr_cols = [c for c in corr_cols if c in df.columns]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, coin in zip(axes, ['usdt', 'usdc']):
    coin_df = df[df['coin'] == coin][corr_cols].dropna()
    corr = coin_df.corr()
    
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
                center=0, square=True, ax=ax, vmin=-1, vmax=1)
    ax.set_title(f'{coin.upper()} Feature Correlations')

plt.tight_layout()
plt.show()

In [ ]:
# Key correlations with price deviation
print("\nCORRELATIONS WITH PRICE DEVIATION")
print("-"*50)

for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin].copy()
    coin_df['price_deviation'] = coin_df['implied_price'] - 1
    
    print(f"\n{coin.upper()}:")
    for col in ['spread_proxy', 'buy_ratio', 'quote_volume', 'circulating_change_pct', 'fear_greed_value']:
        if col in coin_df.columns:
            corr = coin_df['price_deviation'].corr(coin_df[col])
            print(f"  {col:25s}: {corr:+.4f}")

---
## 9. Temporal Patterns

In [ ]:
# Add time features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['price_deviation'] = (df['implied_price'] - 1) * 100

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# By year
ax1 = axes[0, 0]
yearly = df.groupby(['year', 'coin'])['price_deviation'].agg(['mean', 'std']).reset_index()
for coin in ['usdt', 'usdc']:
    coin_yearly = yearly[yearly['coin'] == coin]
    ax1.bar([y + (0.2 if coin == 'usdc' else -0.2) for y in coin_yearly['year']], 
            coin_yearly['mean'], width=0.4, label=coin.upper(), color=colors[coin], alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_xlabel('Year')
ax1.set_ylabel('Mean Deviation (%)')
ax1.set_title('Average Price Deviation by Year')
ax1.legend()

# By month
ax2 = axes[0, 1]
monthly = df.groupby(['month', 'coin'])['price_deviation'].mean().reset_index()
for coin in ['usdt', 'usdc']:
    coin_monthly = monthly[monthly['coin'] == coin]
    ax2.plot(coin_monthly['month'], coin_monthly['price_deviation'], 
             marker='o', label=coin.upper(), color=colors[coin])
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Month')
ax2.set_ylabel('Mean Deviation (%)')
ax2.set_title('Seasonal Pattern (by Month)')
ax2.set_xticks(range(1, 13))
ax2.legend()

# By day of week
ax3 = axes[1, 0]
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow = df.groupby(['day_of_week', 'coin'])['price_deviation'].mean().reset_index()
for coin in ['usdt', 'usdc']:
    coin_dow = dow[dow['coin'] == coin]
    ax3.plot(coin_dow['day_of_week'], coin_dow['price_deviation'], 
             marker='o', label=coin.upper(), color=colors[coin])
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_xlabel('Day of Week')
ax3.set_ylabel('Mean Deviation (%)')
ax3.set_title('Day of Week Pattern')
ax3.set_xticks(range(7))
ax3.set_xticklabels(dow_names)
ax3.legend()

# Volatility by year
ax4 = axes[1, 1]
vol_yearly = df.groupby(['year', 'coin'])['spread_proxy'].mean().reset_index()
for coin in ['usdt', 'usdc']:
    coin_vol = vol_yearly[vol_yearly['coin'] == coin]
    ax4.plot(coin_vol['year'], coin_vol['spread_proxy']*100, 
             marker='s', label=coin.upper(), color=colors[coin], linewidth=2)
ax4.set_xlabel('Year')
ax4.set_ylabel('Avg Spread (%)')
ax4.set_title('Average Daily Spread by Year')
ax4.legend()

plt.tight_layout()
plt.show()

---
## 10. Outlier Detection

In [ ]:
# Identify outliers using IQR method
def find_outliers(series, k=1.5):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return (series < lower) | (series > upper)

print("OUTLIER ANALYSIS (IQR Method, k=1.5)")
print("="*60)

outlier_cols = ['implied_price', 'spread_proxy', 'quote_volume', 'circulating_change_pct']

for coin in ['usdt', 'usdc']:
    print(f"\n{coin.upper()}:")
    coin_df = df[df['coin'] == coin]
    
    for col in outlier_cols:
        if col in coin_df.columns:
            outliers = find_outliers(coin_df[col])
            n_outliers = outliers.sum()
            pct = n_outliers / len(coin_df) * 100
            print(f"  {col:25s}: {n_outliers:4d} outliers ({pct:.2f}%)")

In [ ]:
# Show extreme deviation events
print("\nEXTREME DEVIATION EVENTS (|deviation| > 1%)")
print("="*60)

extreme = df[df['price_deviation'].abs() > 1].copy()
extreme = extreme.sort_values('price_deviation', key=abs, ascending=False)

if len(extreme) > 0:
    display(extreme[['date', 'coin', 'implied_price', 'price_deviation', 
                     'close', 'spread_proxy', 'fear_greed_value']].head(20))
else:
    print("No extreme deviation events found (good stability!)")

---
## 11. Key Findings Summary

In [ ]:
print("="*60)
print("KEY FINDINGS SUMMARY")
print("="*60)

for coin in ['usdt', 'usdc']:
    coin_df = df[df['coin'] == coin]
    deviation = coin_df['implied_price'] - 1
    
    print(f"\n{coin.upper()}:")
    print(f"  • Data period: {coin_df['date'].min().date()} to {coin_df['date'].max().date()}")
    print(f"  • Total observations: {len(coin_df):,}")
    print(f"  • Mean price: ${coin_df['implied_price'].mean():.6f}")
    print(f"  • Price stability (std): {deviation.std()*100:.4f}%")
    print(f"  • Depeg events (>0.5%): {(deviation.abs() > 0.005).sum()} days")
    print(f"  • Supply growth: {coin_df['total_circulating'].iloc[0]/1e9:.1f}B → {coin_df['total_circulating'].iloc[-1]/1e9:.1f}B")
    print(f"  • Avg daily volume: ${coin_df['quote_volume'].mean()/1e6:.0f}M")

# Overall insights
print("\n" + "="*60)
print("OVERALL INSIGHTS:")
print("="*60)
print("• Both stablecoins maintain tight peg (typical deviation < 0.1%)")
print("• USDT has larger market cap and trading volume")
print("• USDC shows occasional larger deviations (e.g., SVB crisis)")
print("• Fear & Greed index shows weak correlation with deviation")
print("• Spread (volatility) increases during market stress periods")